1. Install the required packages

In [ ]:
%pip install tweepy
%pip install pyodbc
%pip install sqlalchemy
%pip install pandas
%pip install python-dotenv

2. Import the installed libraries

In [97]:
import tweepy
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import MetaData, Table, Column, BigInteger, Integer, String
from dotenv import load_dotenv
import os

3. Load Environment Variables

In [77]:
load_dotenv()
api_key = os.getenv("TWITTER_API_KEY")
server_name = os.getenv("SERVER_NAME")
database_name = os.getenv("DATABASE_NAME")

4. Setup a Connection to our Database

In [80]:
server = server_name
database = database_name
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f"mssql+pyodbc://@{server}/{database}?driver={driver.replace(' ', '+')}"

engine = sa.create_engine(connection_string)

5. Create Table for Users

In [98]:
meta = MetaData()

users = Table(
    'users', meta,
    Column('user_id', BigInteger, primary_key=True),
    Column('name', String(50)),
    Column('username', String(50), nullable=False),
    Column('followers_count', Integer),
    Column('following_count', Integer),
    Column('tweet_count', Integer)
)

meta.create_all(engine)

6. Setup Twitter API Client

In [ ]:
client = tweepy.Client(api_key)

7. Retrieve User Info

In [99]:
while True:
    test_input = input("Enter a Twitter handle: ").strip()
    user = client.get_user(username=test_input, user_fields=["public_metrics"])

    if user.data is None:
        print("User not found. Please try again. \n")
        continue
    else:
        print("User found! \n")
        user = user.data
        break

User found! 



8. Store User Info

In [103]:
def user_exists(user_id):
    with engine.connect() as conn:
        query = sa.select(users).where(users.c.user_id == int(user.id))
        result = conn.execute(query).fetchone()
        return result is not None
    
if user_exists(user.id):
    print("User already exists in the database. \n")
else:
    new_user = {
        'user_id': int(user.id),
        'name': user.name,
        'username': user.username,
        'followers_count': int(user.public_metrics['followers_count']),
        'following_count': int(user.public_metrics['following_count']),
        'tweet_count': int(user.public_metrics['tweet_count'])
    }

    with engine.begin() as conn:
        conn.execute(users.insert().values(new_user))
        print("User data inserted into the database. \n")

User already exists in the database. 

